In [2]:
from optics import Point, Optics
import psycopg2
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins
import fiona

In [37]:
import matplotlib.cm as cmx
import matplotlib.colors as colors


def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b


def style_function(feature, n_colors):
    cid = feature['properties']['cid']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#red' if cid is None else "rgb{}".format(rgb(0, n_colors, cid))
    }

def init_style_function(n_colors):
    return lambda feature: style_function(feature, n_colors)

In [8]:
rapperswil_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1683921"
zurich_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682248"
zuerich_small_polygon_query = "SELECT ST_Buffer(ST_Transform(ST_SetSrid(ST_MakePoint(8.5405, 47.3752), 4326), 3857), 500)"
switzerland_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -51701"
hombrechtikon_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682143"
staefa_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682214"

rapperswil_location = [47.226, 8.818]
zurich_location = [47.3763, 8.5403]
hombrechtikon_location = [47.25212, 8.77038]
staefa_location = [47.24062, 8.72303]

epsg = 3857

polygon_query = rapperswil_polygon_query
location = rapperswil_location

In [50]:
query = """
SELECT ST_Centroid(geometry) AS geometry FROM pois WHERE ST_Intersects(geometry, ({polygon}));
""".format(polygon=polygon_query)

with psycopg2.connect("") as conn:
    pois = gpd.read_postgis(query, conn, geom_col='geometry')

pois.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(pois).add_to(m)

m

In [51]:
points = [Point(poi.y, poi.x) for poi in pois.to_crs({'init': 'epsg:4326'}).geometry]

optics = Optics(points, 100, 3) # 100m radius for neighbor consideration, cluster size >= 2 points
optics.run()                    # run the algorithm
clusters = optics.cluster(100) # 50m threshold for clustering

insert_query = """
INSERT INTO pois_optics(cid, geometry)
VALUES({cluster_id}, ST_Transform((ST_SetSrid(ST_MakePoint({longitude}, {latitude}), 4326)), 3857))
"""


inserts = []
i = 0
for cluster in clusters:
    inserts += [insert_query.format(cluster_id=i, longitude=point.longitude, latitude=point.latitude) for point in cluster.points]
    i += 1
    
    
with psycopg2.connect("") as conn:
    with conn.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS pois_optics; CREATE TABLE pois_optics(cid integer, geometry geometry)")
        cursor.execute(";".join(inserts))

In [49]:
query = """
SELECT ST_ConvexHull(ST_Union(geometry)) AS geometry, cid FROM pois_optics GROUP BY cid;
"""

with psycopg2.connect("") as conn:
    pois = gpd.read_postgis(query, conn, geom_col='geometry')

pois.crs = fiona.crs.from_epsg(3857)
n_clusters = len(pois.groupby('cid').cid.nunique())

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)
folium.GeoJson(pois, style_function=init_style_function(n_clusters)).add_to(m)


m

In [70]:
points = [Point(poi.y, poi.x) for poi in pois.geometry]

optics = Optics(points, 100, 3) # 100m radius for neighbor consideration, cluster size >= 2 points
optics.run()                    # run the algorithm
clusters = optics.cluster(50) # 50m threshold for clustering

m = folium.Map(location=location, zoom_start=16, tiles="cartodbpositron")

folium.plugins.Fullscreen().add_to(m)

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
             'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
             'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
             'gray', 'black', 'lightgray']

i = 0
for cluster in clusters:
    color = colors[min(i, len(colors) - 1)]
    i += 1
    for point in cluster.points:
        folium.Marker([point.latitude, point.longitude],
                  icon=folium.Icon(color=color),
                  ).add_to(m)

m